In [ ]:
## This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
        
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import numpy as np
import pandas as pd

import pydicom as dicom
import os
import cv2
import PIL # optional
import shutil

In [ ]:
# Image preprocessing 

In [ ]:
D_type = pd.read_csv('/kaggle/input/decease-type-files/decease_type.csv')

In [ ]:
D_type

In [ ]:
!mkdir Bbox_256x256

In [ ]:
crp_file_name = []
for i in np.arange(D_type.shape[0]):
    r = D_type.iloc[i]
    im1 = tf.keras.preprocessing.image.load_img(r.filename)
    ary = tf.keras.preprocessing.image.img_to_array(im1)
    jt =tf.keras.preprocessing.image.array_to_img(ary[r.ymin:r.ymax,r.xmin:r.xmax]) # [ymin:Ymax,xmin:xmax]
    hp = jt.resize((256,256))
    hp.save('/kaggle/working/Bbox_256x256/'+ f'{r.xmin}_'+ f'{r.ymin}_'+f'{r.xmax}_'+ f'{r.ymax}'+'.png')
    crp_file_name.append('/kaggle/working/Bbox_256x256/'+ f'{r.xmin}_'+ f'{r.ymin}_'+f'{r.xmax}_'+ f'{r.ymax}'+'.png')

In [ ]:
D_type['crp_file_name'] = crp_file_name
D_type

In [ ]:
 ! zip "/kaggle/working/Bbox_256x256.zip" "/kaggle/working/" 

In [ ]:
D_type[['crp_file_name','class']]

In [ ]:
pd.get_dummies(D_type[['class']])

In [ ]:
# Just taking only the required columns from the trainstudy dataframe
kj = D_type[['crp_file_name','class']]
kt = pd.get_dummies(D_type[['class']])
fea = pd.concat([kt,kj],axis=1)
fea.columns = ['Atypical', 'Indeterminate','Typical', 'crp_file_name','class']
fea

In [ ]:
from sklearn.model_selection import train_test_split

X_trn,X_val,Y_trn,Y_val = train_test_split(fea,fea[['class']],test_size=0.15,random_state=45)

#X_trn
X_val

# Pre-Trained Model Activation 

In [ ]:
img_size = 256
img_depth = 3
model = tf.keras.applications.efficientnet.EfficientNetB7(include_top=False, #Do not include FC layer at the end
                                          input_shape=(img_size,img_size, img_depth),
                                          weights='imagenet')

In [ ]:
#Set pre-trained model layers to not trainable
for layer in model.layers:
    layer.trainable = False

In [ ]:
#get Output layer of Pre0trained model
x = model.output

#Flatten the output to feed to Dense layer
x = tf.keras.layers.Flatten()(x)

#Add one Dense layer
x = tf.keras.layers.Dense(200, activation='relu')(x)

#Add output layer
prediction = tf.keras.layers.Dense(3,activation='softmax')(x)

In [ ]:
#Using Keras Model class
final_model = tf.keras.models.Model(inputs=model.input, #Pre-trained model input as input layer
                                    outputs=prediction) #Output layer added

In [ ]:
final_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Image data augumentataion 
trn_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255.,
                                                fill_mode="nearest",horizontal_flip=True,
                                                #validation_split=0.2,
                                                preprocessing_function=tf.keras.applications.efficientnet.preprocess_input)


val_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255.,
                                                fill_mode="nearest",horizontal_flip=True,
                                                #validation_split=0.2,
                                                preprocessing_function=tf.keras.applications.efficientnet.preprocess_input)

test_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255.)


In [ ]:
X_trn

In [ ]:
#Create train and test generator
batchsize = 128
train_generator =  trn_gen.flow_from_dataframe (X_trn,
                                                x_col="crp_file_name",
                                                y_col=['Atypical', 'Indeterminate', 'Typical'],
                                                target_size=(256, 256),
                                                color_mode="rgb",
                                            #classes=['Negative','Typical','Indeterminate','Atypical'], 
                                                class_mode="raw",
                                                batch_size=batchsize,
                                                shuffle=True,
                                                seed=20,
                                                ) #batchsize can be changed

val_generator = val_gen.flow_from_dataframe (X_val,
                                            x_col="crp_file_name",
                                            y_col=['Atypical', 'Indeterminate','Typical'],
                                            target_size=(256, 256),
                                            color_mode="rgb",
                                            #classes=['Negative','Typical','Indeterminate','Atypical'], 
                                            class_mode="raw",
                                            batch_size=batchsize,
                                            shuffle=True,
                                            seed=20,
                                            )

In [ ]:
final_model.fit_generator(train_generator, 
                          epochs=10,
                          steps_per_epoch= X_trn.shape[0]//batchsize,
                          validation_data=val_generator,
                          validation_steps = X_val.shape[0]//batchsize,
                          verbose=1)

In [ ]:
'''
print(len(model.layers))
for layer in model.layers[150:]:
    layer.trainable =  True 
'''